# Programmatically retrieving information about simulation tools registered with BioSimulators

[BioSimulators](https://biosimulators.org) contains extensive information about simulation software tools. This includes information about the model formats (e.g., CellML, SBML), modeling frameworks (e.g., flux balance, logical, ordinary differential equations), simulation types (e.g., steady-state, time course), and algorithms (e.g., CVODE, SSA) supported by each tool. This information can be helpful for finding an appropriate simulation tool for a specific project and for using simulation tools, especially with the [Simulation Experiment Description Markup Langauge (SED-ML)](http://sed-ml.org/).

This information can be retrieved using [BioSimulators' REST API](https://api.biosimulators.org). This API returns information about simulation tools in [JavaScript Object Notation (JSON)](https://www.json.org/) format. The schemas that BioSimulators uses are described in the  [documentation](https://api.biosimulators.org) for the API. The API follows the [OpenAPI](https://swagger.io/specification/) standards.

These schemas utilize several ontologies:
* Funding agencies: [FunderRegistry](https://www.crossref.org/services/funder-registry/)
* Modeling frameworks: [Systems Biology Ontology (SBO)](https://wwwdev.ebi.ac.uk/sbo/)
* Modeling formats: [EDAM](https://edamontology.org/page) and [SED-ML model language URNs](https://sed-ml.org/urns.html)
* Output dimensions: [Semanticscience Integrated Ontology (SIO)](https://semanticscience.org/)
* Outputs of implicit simulation variables: [SED-ML symbol URNs](https://sed-ml.org/urns.html)
* Programming languages: [Linguist](https://github.com/github/linguist)
* Simulation algorithms, their parameters, and their outputs: [Kinetic Simulation Algorithm Ontology (KiSAO)](https://github.com/SED-ML/KiSAO)
* Software licenses: [Software Package Data Exchange (SPDX)](https://spdx.org/licenses)

This tutorial illustrates how to programmatically retrieve information about simulation tools for BioSimulators' REST API. Please see the other tools for more information about using simulation tools to execute individual simulations and entire simulation projects.

## 1. Retrieve a list of the simulation tools registered with BioSimulators

Import the [`requests`](https://docs.python-requests.org/en/latest/) package to run HTTP requests to BioSimulators' API.

In [1]:
import requests

The https://api.biosimulators.org/simulators/latest endpoint provides a list of all of the registered simulation tools and information about their most recent versions. Next, use the requests package to query this endpoint.

In [2]:
response = requests.get('https://api.biosimulators.org/simulators/latest')

Run the `raise_for_status` method to check that the request succeeded.

In [3]:
response.raise_for_status()

Call the `json` method to get the data returned by BioSimulators' API.

In [4]:
simulators = {simulator['id']: simulator for simulator in response.json()}

Get one simulation tool.

In [5]:
simulator = simulators['cobrapy']

Print out information about the tool

In [6]:
import yaml
print(yaml.dump(simulator))

algorithms:
- archiveFormats:
  - id: format_3686
    namespace: EDAM
    supportedFeatures: []
    version: null
  availableSoftwareInterfaceTypes:
  - library
  - command-line application
  - BioSimulators Docker image
  citations:
  - authors: "Jeffrey D. Orth, Ines Thiele & Bernhard \xD8. Palsson"
    identifiers:
    - id: 10.1038/nbt.1614
      namespace: doi
      url: https://doi.org/10.1038/nbt.1614
    issue: '3'
    journal: Nature Biotechnology
    pages: 245-248
    title: What is flux balance analysis?
    volume: '28'
    year: 2010
  dependencies:
  - freeNonCommercialLicense: true
    name: GLPK
    required: false
    url: https://www.gnu.org/software/glpk/
    version: null
  - freeNonCommercialLicense: true
    name: CPLEX
    required: false
    url: https://www.ibm.com/analytics/cplex-optimizer
    version: null
  - freeNonCommercialLicense: true
    name: Gurobi
    required: false
    url: https://www.gurobi.com/
    version: null
  id: fba
  kisaoId:
    id: KI

## 2. Find the validated simulation tools that provide Python APIs

In [7]:
simulators_with_apis = {}
for id, simulator in simulators.items():
    if simulator['biosimulators']['validated'] and simulator.get('pythonApi', None):
        simulators_with_apis[id] = simulator

print(sorted(simulators_with_apis.keys()))

['amici', 'bionetgen', 'boolnet', 'brian2', 'cbmpy', 'cobrapy', 'copasi', 'gillespy2', 'ginsim', 'libsbmlsim', 'masspy', 'netpyne', 'neuron', 'opencor', 'pyneuroml', 'pysces', 'rbapy', 'smoldyn', 'tellurium', 'xpp']


## 3. Find the simulation tools that support a particular model format (e.g. SBML, [EDAM:format_2585](https://www.ebi.ac.uk/ols/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fformat_2585))

In [8]:
sbml_simulators = {}
for id, simulator in simulators_with_apis.items():
    for algorithm in simulator['algorithms']:
        for model_format in algorithm['modelFormats']:
            if model_format['id'] == 'format_2585':
                sbml_simulators[id] = simulator
                
print(sorted(sbml_simulators.keys()))

['amici', 'boolnet', 'cbmpy', 'cobrapy', 'copasi', 'gillespy2', 'ginsim', 'libsbmlsim', 'masspy', 'pysces', 'tellurium']


## 4. Find the simulation tools that support a particular modeling framework (e.g. ODE, [SBO:0000293](https://www.ebi.ac.uk/sbo/main/SBO:0000293))

In [9]:
ode_simulators = {}
for id, simulator in simulators_with_apis.items():
    for algorithm in simulator['algorithms']:
        for model_format in algorithm['modelingFrameworks']:
            if model_format['id'] == 'SBO_0000293':
                ode_simulators[id] = simulator
                
print(sorted(ode_simulators.keys()))

['amici', 'bionetgen', 'brian2', 'copasi', 'gillespy2', 'libsbmlsim', 'masspy', 'netpyne', 'neuron', 'opencor', 'pyneuroml', 'pysces', 'tellurium', 'xpp']


## 5. Find the simulation tools that support a particular type of simulation or analysis (e.g. SED-ML uniform time course, `SedUniformTimeCourseSimulation`)

In [10]:
time_course_simulators = {}
for id, simulator in simulators_with_apis.items():
    for algorithm in simulator['algorithms']:
        if 'SedUniformTimeCourseSimulation' in algorithm['simulationTypes']:
            time_course_simulators[id] = simulator
                
print(sorted(time_course_simulators.keys()))

['amici', 'bionetgen', 'boolnet', 'brian2', 'copasi', 'gillespy2', 'ginsim', 'libsbmlsim', 'masspy', 'netpyne', 'neuron', 'opencor', 'pyneuroml', 'pysces', 'smoldyn', 'tellurium', 'xpp']


## 6. Find the simulation tools that support a particular algorithm (e.g. CVODE, [KISAO_0000019](https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000019))

In [11]:
cvode_simulators = {}
for id, simulator in simulators_with_apis.items():
    for algorithm in simulator['algorithms']:
        if algorithm['kisaoId']['id'] == 'KISAO_0000019':
            cvode_simulators[id] = simulator
                
print(sorted(cvode_simulators.keys()))

['bionetgen', 'masspy', 'netpyne', 'neuron', 'opencor', 'pysces', 'tellurium', 'xpp']


## 7. Get a specific simulation tool such as [COPASI](http://copasi.org/)

In [12]:
copasi = simulators['copasi']

## 8. Get the algorithms implemented by the tool

In [13]:
algorithms = {}
for algorithm in copasi['algorithms']:
    kisao_id = algorithm['kisaoId']['id']
    algorithms[kisao_id] = algorithm
print(sorted(algorithms.keys()))

['KISAO_0000027', 'KISAO_0000029', 'KISAO_0000039', 'KISAO_0000048', 'KISAO_0000304', 'KISAO_0000560', 'KISAO_0000561', 'KISAO_0000562', 'KISAO_0000563', 'KISAO_0000566']


## 9. Get a specific algorithm implemented by the tool, such as LSODA, [KISAO_0000560](https://www.ebi.ac.uk/ols/ontologies/kisao/terms?iri=http%3A%2F%2Fwww.biomodels.net%2Fkisao%2FKISAO%23KISAO_0000560))

In [14]:
lsoda = algorithms['KISAO_0000560']

## 10. Get additional information about the algorithm from the KiSAO ontology

In [15]:
from kisao import Kisao
kisao = Kisao()
term = kisao.get_term(lsoda['kisaoId']['id'])
print(term.name)

LSODA/LSODAR hybrid method


## 11. Get the parameters of the algorithm and their names, data types, and default values

In [16]:
parameters = {}
for parameter in lsoda['parameters']:
    parameters[parameter['kisaoId']['id']] = {
        'name': kisao.get_term(parameter['kisaoId']['id']).name,
        'type': parameter['type'],
        'value': parameter['value'],
    }
print(yaml.dump(parameters))

KISAO_0000209:
  name: relative tolerance
  type: float
  value: 1e-6
KISAO_0000211:
  name: absolute tolerance
  type: float
  value: 1e-12
KISAO_0000216:
  name: use reduced model
  type: boolean
  value: 'true'
KISAO_0000415:
  name: maximum number of steps
  type: integer
  value: '10000'
KISAO_0000467:
  name: maximum step size
  type: float
  value: null



## 12. Get the model changes supported by the implementation of the algorithm by the tool

In [17]:
print(yaml.dump(algorithm['modelChangePatterns']))

- name: Change component attributes
  symbol: null
  target:
    grammar: XPath
    value: //*/@*
  types:
  - SedAttributeModelChange
  - SedComputeAttributeChangeModelChange
  - SedSetValueAttributeModelChange
- name: Add components
  symbol: null
  target:
    grammar: XPath
    value: //*
  types:
  - SedAddXmlModelChange
- name: Remove components
  symbol: null
  target:
    grammar: XPath
    value: //*
  types:
  - SedRemoveXmlModelChange
- name: Change components
  symbol: null
  target:
    grammar: XPath
    value: //*
  types:
  - SedChangeXmlModelChange



## 13. Get the observables supported by the implementation of the algorithm by the tool

In [18]:
print(yaml.dump(algorithm['outputVariablePatterns']))

- name: time
  symbol:
    namespace: urn:sedml:symbol
    value: time
  target: null
- name: species concentrations
  symbol: null
  target:
    grammar: XPath
    value: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species
- name: reaction fluxes
  symbol: null
  target:
    grammar: XPath
    value: /sbml:sbml/sbml:model/sbml:listOfReactions/sbml:reaction
- name: compartment volumes
  symbol: null
  target:
    grammar: XPath
    value: /sbml:sbml/sbml:model/sbml:listOfCompartments/sbml:compartment
- name: parameter values
  symbol: null
  target:
    grammar: XPath
    value: /sbml:sbml/sbml:model/sbml:listOfParameters/sbml:parameter



## 14. Get more information about model formats and modeling frameworks

Information about all of the ontologies used by BioSimulators is available from [BioSimulations' ontology REST API](https://api.biosimulations.org/). Documentation is available inline at this URL. The API can be queried as illustrated below to retrieve information about specific terms, such as the Systems Biology Markup Language (SBML) ([EDAM:format_2585](https://www.ebi.ac.uk/ols/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fformat_2585)) and ordinary differential equations (ODES) ([SBO_0000293](https://www.ebi.ac.uk/ols/ontologies/sbo/terms?iri=http%3A%2F%2Fbiomodels.net%2FSBO%2FSBO_0000293)).

In [19]:
response = requests.get('https://api.biosimulations.org/ontologies/EDAM/format_2585')
response.raise_for_status()
print(yaml.dump(response.json()))

biosimulationsMetadata:
  acronym: SBML
  icon: model
  modelFormatMetadata:
    introspectionAvailable: true
    sedUrn: urn:sedml:language:sbml
    validationAvailable: true
  omexManifestUris:
  - http://identifiers.org/combine.specifications/sbml
children: []
description: Systems Biology Markup Language (SBML), the standard XML format for models
  of biological processes such as for example metabolism, cell signaling, and gene
  regulation.
fileExtensions:
- xml
- sbml
id: format_2585
iri: http://edamontology.org/format_2585
mediaTypes:
- application/sbml+xml
- application/x-sbml
- application/xml
moreInfoUrl: http://sbml.org
name: SBML
namespace: EDAM
parents:
- format_2013
- format_2332
url: https://www.ebi.ac.uk/ols/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fformat_2585



In [20]:
response = requests.get('https://api.biosimulations.org/ontologies/SBO/SBO_0000293')
response.raise_for_status()
print(yaml.dump(response.json()))

children:
- SBO_0000675
- SBO_0000676
description: 'Modelling approach where the quantities of participants are considered
  continuous, and represented by real values. The associated simulation methods make
  use of differential equations. The models do not take into account the distribution
  of the entities and describe only the temporal fluxes.  '
id: SBO_0000293
iri: http://biomodels.net/SBO/SBO_0000293
moreInfoUrl: null
name: non-spatial continuous framework
namespace: SBO
parents:
- SBO_0000062
url: https://www.ebi.ac.uk/ols/ontologies/sbo/terms?iri=http%3A%2F%2Fbiomodels.net%2FSBO%2FSBO_0000293



In [21]:
import importlib
api = importlib.import_module(copasi['pythonApi']['module'])

## 16. Use the API for the simulator to get its version and the available version of the tool

In [22]:
print(api.__version__)
print(api.get_simulator_version())

0.1.34
4.34.251
